**Beyond text classification: Sequence-to-sequence learning**

In [4]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2022-12-06 14:17:09--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.128, 173.194.79.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.01s   

2022-12-06 14:17:09 (189 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [5]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [6]:
import random
print(random.choice(text_pairs))

('I like coffee better.', '[start] Prefiero el café. [end]')


In [7]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [8]:
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [9]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [10]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [11]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

In [12]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [13]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 135s 92ms/step - loss: 1.6335 - accuracy: 0.4172 - val_loss: 1.3199 - val_accuracy: 0.5067
Epoch 2/15
1302/1302 [==============================] - 116s 89ms/step - loss: 1.3124 - accuracy: 0.5278 - val_loss: 1.1589 - val_accuracy: 0.5698
Epoch 3/15
1302/1302 [==============================] - 116s 89ms/step - loss: 1.1696 - accuracy: 0.5776 - val_loss: 1.0840 - val_accuracy: 0.5989
Epoch 4/15
1302/1302 [==============================] - 116s 89ms/step - loss: 1.0788 - accuracy: 0.6099 - val_loss: 1.0462 - val_accuracy: 0.6192
Epoch 5/15
1302/1302 [==============================] - 116s 89ms/step - loss: 1.0314 - accuracy: 0.6336 - val_loss: 1.0318 - val_accuracy: 0.6304
Epoch 6/15
1302/1302 [==============================] - 115s 88ms/step - loss: 1.0010 - accuracy: 0.6520 - val_loss: 1.0274 - val_accuracy: 0.6348
Epoch 7/15
1302/1302 [==============================] - 115s 89ms/step - loss: 0.9813 - accuracy: 0.6655 - val_loss: 1

In [14]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Where is the problem?
1/1 [==============================] - 0s 21ms/step
[start] dónde está el problema [end]
-
You're back.
1/1 [==============================] - 0s 21ms/step
[start] estás de vuelta [end]
-
By this time next year, you will have visited almost all the famous places in Japan.
1/1 [==============================] - 0s 22ms/step
[start] para este tiempo el que viene más pronto el año pasado [end]
-
At least they listened to me.
1/1 [==============================] - 0s 23ms/step
[start] al menos me lo son [end]
-
He looked her right in the eye.
1/1 [==============================] - 0s 22ms/step
[start] Él la quiero bien en la los ojos [end]
-
He was too curious.
1/1 [==============================] - 0s 22ms/step
[start] Él era demasiado al Él [end]
-
Don't ask me why.
1/1 [==============================] - 0s 20ms/step
[start] no me hagas por qué [end]
-
Do what Tom says.
1/1 [==============================] - 0s 20ms/step
[start] lo que tom dice [end]
-
It's a pity

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [16]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [17]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        # Multi-Head Attention层
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        # Feed Forward层
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        # Add&Norm层1
        self.layernorm_1 = layers.LayerNormalization()
        # Add&Norm层2
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs):
        # 首先经过Multi-Head Attention层
        attention_output = self.attention(inputs, inputs)
        # 经过Add&Norm层1
        proj_input = self.layernorm_1(inputs + attention_output)
        # 经过Feed Forward层
        proj_output = self.dense_proj(proj_input)
        # 经过Add&Norm层2
        return self.layernorm_2(proj_input + proj_output)
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 94s 70ms/step - loss: 1.7673 - accuracy: 0.3811 - val_loss: 1.4460 - val_accuracy: 0.4719
Epoch 2/30
1302/1302 [==============================] - 90s 69ms/step - loss: 1.4052 - accuracy: 0.5047 - val_loss: 1.2784 - val_accuracy: 0.5329
Epoch 3/30
1302/1302 [==============================] - 89s 68ms/step - loss: 1.2388 - accuracy: 0.5609 - val_loss: 1.1337 - val_accuracy: 0.5826
Epoch 4/30
1302/1302 [==============================] - 89s 68ms/step - loss: 1.1355 - accuracy: 0.5981 - val_loss: 1.0867 - val_accuracy: 0.6080
Epoch 5/30
1302/1302 [==============================] - 90s 69ms/step - loss: 1.0826 - accuracy: 0.6217 - val_loss: 1.0600 - val_accuracy: 0.6206
Epoch 6/30
1302/1302 [==============================] - 90s 69ms/step - loss: 1.0510 - accuracy: 0.6385 - val_loss: 1.0520 - val_accuracy: 0.6254
Epoch 7/30
1302/1302 [==============================] - 90s 69ms/step - loss: 1.0289 - accuracy: 0.6511 - val_loss: 1.0350 -

In [19]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom's field of study is law.
[start] el campo de tom [UNK] se ha pasado [end]
-
He could ski well when he was young.
[start] no pudo [UNK] cuando era joven [end]
-
We have something very important that we need to discuss.
[start] tenemos algo muy importante es que necesitamos hablar [end]
-
Will he come this evening?
[start] se va a venir esta tarde [end]
-
I wanted to kill Tom.
[start] quería matar a tom [end]
-
The baby is crawling.
[start] el bebé es el [UNK] [end]
-
She told him that she was happy.
[start] ella le dijo que era feliz [end]
-
He bowed to me as he left the room.
[start] me rompió como cuando él salió de la habitación [end]
-
What is the time?
[start] qué es la hora [end]
-
Coffee prices have jumped almost 50% in six months.
[start] los seis [UNK] van casi tres niños a mes meses [end]
-
Look at that pole in the square.
[start] mira eso tiene un [end]
-
This is a very old book.
[start] este es un libro muy viejo [end]
-
Don't you feel stupid now?
[start] no te sientes